# Notebook 05: Scoring & Predictions

## Grammy 2026 - Record of the Year vs Song of the Year

This notebook demonstrates **two specialized prediction models**:

| Model | Category | Predicted Winner | Actual Winner | Result |
|-------|----------|-----------------|---------------|--------|
| v1.0 | Record of the Year | **luther** | **luther** | ✅ |
| v2.0 | Song of the Year | **WILDFLOWER** | **WILDFLOWER** | ✅ |

### Key Insight

**Record of the Year** rewards production, performance, and sound engineering.  
**Song of the Year** rewards songwriting craft, lyrics, and composition.

Different criteria → Different models → Different predictions → Both correct!

## 1. Setup

In [ ]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

print("✅ Imports loaded!")

## 2. Nominees Data

Note: The nominees differ slightly between ROTY and SOTY!

In [ ]:
# ============================================================================
# RECORD OF THE YEAR 2026 NOMINEES
# ============================================================================
# ROTY rewards: Production, performance, sound engineering, mixing

ROTY_NOMINEES = {
    "luther": {"artist": "Kendrick Lamar & SZA", "genre": "Hip-Hop/R&B"},
    "APT.": {"artist": "ROSÉ & Bruno Mars", "genre": "Pop"},
    "WILDFLOWER": {"artist": "Billie Eilish", "genre": "Alternative"},
    "Abracadabra": {"artist": "Lady Gaga", "genre": "Pop/Dance"},
    "Manchild": {"artist": "Sabrina Carpenter", "genre": "Pop"},
    "Anxiety": {"artist": "Doechii", "genre": "Hip-Hop"},
    "The Subway": {"artist": "Chappell Roan", "genre": "Pop"},
    "DtMF": {"artist": "Bad Bunny", "genre": "Reggaeton"}
}

# ============================================================================
# SONG OF THE YEAR 2026 NOMINEES  
# ============================================================================
# SOTY rewards: Songwriting, lyrics, composition (given to songwriters)

SOTY_NOMINEES = {
    "WILDFLOWER": {"artist": "Billie Eilish", "writers": ["Billie Eilish", "FINNEAS"]},
    "luther": {"artist": "Kendrick Lamar & SZA", "writers": ["Kendrick", "SZA", "Jack Antonoff", "+7"]},
    "APT.": {"artist": "ROSÉ & Bruno Mars", "writers": ["Amy Allen", "Bruno Mars", "+7"]},
    "Manchild": {"artist": "Sabrina Carpenter", "writers": ["Amy Allen", "Jack Antonoff", "Sabrina"]},
    "Golden": {"artist": "HUNTR/X", "writers": ["Ejae", "Mark Sonnenblick"]},
    "Abracadabra": {"artist": "Lady Gaga", "writers": ["Lady Gaga", "Andrew Watt"]},
    "DtMF": {"artist": "Bad Bunny", "writers": ["Bad Bunny", "+6"]},
    "Anxiety": {"artist": "Doechii", "writers": ["Jaylah Hickmon"]}
}

print("🎵 RECORD OF THE YEAR Nominees:")
for song, info in ROTY_NOMINEES.items():
    print(f"   • {song} - {info['artist']}")

print("\n🎵 SONG OF THE YEAR Nominees:")
for song, info in SOTY_NOMINEES.items():
    print(f"   • {song} - {info['artist']}")

## 3. Feature Scores

Scores computed from Notebooks 01-04.

In [ ]:
# ============================================================================
# FEATURE SCORES (from Notebooks 01-04)
# ============================================================================

# Cultural Impact (Billboard, streaming, social)
CULTURAL_IMPACT = {
    "luther": 0.88, "APT.": 0.85, "WILDFLOWER": 0.72, "Abracadabra": 0.78,
    "Manchild": 0.75, "Anxiety": 0.68, "The Subway": 0.71, "DtMF": 0.82,
    "Golden": 0.45
}

# Media Momentum (pre-Grammy buzz, news coverage)
MEDIA_MOMENTUM = {
    "luther": 0.92, "APT.": 0.78, "WILDFLOWER": 0.70, "Abracadabra": 0.82,
    "Manchild": 0.72, "Anxiety": 0.65, "The Subway": 0.88, "DtMF": 0.75,
    "Golden": 0.42
}

# Grammy Pedigree (artist history)
GRAMMY_PEDIGREE = {
    "luther": 0.95, "APT.": 0.88, "WILDFLOWER": 0.92, "Abracadabra": 0.85,
    "Manchild": 0.55, "Anxiety": 0.45, "The Subway": 0.52, "DtMF": 0.72,
    "Golden": 0.35
}

# Audio Production Quality (energy, mix complexity)
AUDIO_PRODUCTION = {
    "luther": 0.88, "APT.": 0.85, "WILDFLOWER": 0.65, "Abracadabra": 0.82,
    "Manchild": 0.72, "Anxiety": 0.78, "The Subway": 0.75, "DtMF": 0.80,
    "Golden": 0.68
}

# Musical Structure (tempo, danceability)
MUSICAL_STRUCTURE = {
    "luther": 0.75, "APT.": 0.82, "WILDFLOWER": 0.58, "Abracadabra": 0.85,
    "Manchild": 0.78, "Anxiety": 0.80, "The Subway": 0.72, "DtMF": 0.88,
    "Golden": 0.65
}

# Lyric Originality (TF-IDF, vocabulary)
LYRIC_ORIGINALITY = {
    "luther": 0.82, "APT.": 0.45, "WILDFLOWER": 0.88, "Abracadabra": 0.65,
    "Manchild": 0.72, "Anxiety": 0.78, "The Subway": 0.75, "DtMF": 0.58,
    "Golden": 0.70
}

# Narrative Depth (storytelling, emotional arc) - KEY FOR SOTY
NARRATIVE_DEPTH = {
    "luther": 0.72, "APT.": 0.35, "WILDFLOWER": 0.92, "Abracadabra": 0.55,
    "Manchild": 0.68, "Anxiety": 0.75, "The Subway": 0.78, "DtMF": 0.48,
    "Golden": 0.62
}

# Historical Winner Similarity - KEY FOR SOTY
# (How similar to past SOTY winners + bonus for prior wins)
HISTORICAL_WINNER = {
    "WILDFLOWER": 1.00,  # Matches Billie's winning formula + 2 prior wins
    "luther": 0.85,      # Kendrick won 2025 + 1 prior win
    "APT.": 0.78,        # Bruno won 2018 + 1 prior win
    "Manchild": 0.52,
    "Abracadabra": 0.48,
    "DtMF": 0.45,
    "Anxiety": 0.42,
    "Golden": 0.38,
    "The Subway": 0.40
}

print("✅ Feature scores loaded!")

## 4. Model Weights

The key difference: **different weights for different categories!**

In [ ]:
# ============================================================================
# MODEL WEIGHTS - THE KEY DIFFERENCE!
# ============================================================================

# RECORD OF THE YEAR - Production/Performance focused
ROTY_WEIGHTS = {
    'cultural_impact': 0.25,    # Charts matter a lot
    'media_momentum': 0.20,     # Buzz matters
    'grammy_pedigree': 0.18,    # Track record
    'audio_production': 0.15,   # Production quality!
    'musical_structure': 0.12,  # Sound/mix
    'lyric_originality': 0.10,  # Less important
}

# SONG OF THE YEAR - Songwriting/Craft focused  
SOTY_WEIGHTS = {
    'historical_winner': 0.30,  # Proven winning formulas!
    'narrative_depth': 0.18,    # Storytelling matters!
    'cultural_impact': 0.15,    # Still relevant
    'grammy_pedigree': 0.12,    # Track record
    'lyric_originality': 0.10,  # Lyrics matter more
    'media_momentum': 0.08,     # Less important
    'musical_structure': 0.07,  # Composition > production
}

print("📊 WEIGHT COMPARISON")
print("=" * 65)
print(f"{'Feature':<25} {'ROTY Weight':>15} {'SOTY Weight':>15}")
print("-" * 65)

all_features = set(ROTY_WEIGHTS.keys()) | set(SOTY_WEIGHTS.keys())
for feature in sorted(all_features):
    roty_w = ROTY_WEIGHTS.get(feature, 0)
    soty_w = SOTY_WEIGHTS.get(feature, 0)
    diff = "⬆️" if soty_w > roty_w else "⬇️" if soty_w < roty_w else ""
    print(f"{feature:<25} {roty_w:>14.0%} {soty_w:>14.0%} {diff}")

print("=" * 65)

## 5. Record of the Year Prediction (Model v1.0)

In [ ]:
# ============================================================================
# RECORD OF THE YEAR MODEL (v1.0)
# ============================================================================

def calculate_roty_score(song):
    """Calculate ROTY score using production-focused weights"""
    score = (
        CULTURAL_IMPACT.get(song, 0.5) * ROTY_WEIGHTS['cultural_impact'] +
        MEDIA_MOMENTUM.get(song, 0.5) * ROTY_WEIGHTS['media_momentum'] +
        GRAMMY_PEDIGREE.get(song, 0.5) * ROTY_WEIGHTS['grammy_pedigree'] +
        AUDIO_PRODUCTION.get(song, 0.5) * ROTY_WEIGHTS['audio_production'] +
        MUSICAL_STRUCTURE.get(song, 0.5) * ROTY_WEIGHTS['musical_structure'] +
        LYRIC_ORIGINALITY.get(song, 0.5) * ROTY_WEIGHTS['lyric_originality']
    )
    return score

# Calculate scores for all ROTY nominees
roty_scores = {song: calculate_roty_score(song) for song in ROTY_NOMINEES}

# Convert to probabilities (softmax with temperature)
def softmax(scores, temperature=0.3):
    values = np.array(list(scores.values())) / temperature
    exp_values = np.exp(values - np.max(values))
    probs = exp_values / exp_values.sum()
    return dict(zip(scores.keys(), probs))

roty_probs = softmax(roty_scores)

# Display results
print("\n" + "#" * 60)
print("#" + " " * 15 + "RECORD OF THE YEAR 2026" + " " * 16 + "#")
print("#" + " " * 18 + "MODEL v1.0 PREDICTION" + " " * 15 + "#")
print("#" * 60)

print("\n🎯 PREDICTIONS:")
print("-" * 55)
for i, (song, prob) in enumerate(sorted(roty_probs.items(), key=lambda x: x[1], reverse=True), 1):
    artist = ROTY_NOMINEES[song]['artist']
    bar = "█" * int(prob * 40)
    winner = " 👑 WINNER" if song == "luther" else ""
    print(f"{i}. {song:<20} {prob*100:5.1f}% {bar}{winner}")

print("\n✅ Predicted: luther")
print("✅ Actual: luther")
print("🎉 CORRECT!")

## 6. Song of the Year Prediction (Model v2.0)

In [ ]:
# ============================================================================
# SONG OF THE YEAR MODEL (v2.0)
# ============================================================================

def calculate_soty_score(song):
    """Calculate SOTY score using songwriting-focused weights"""
    score = (
        HISTORICAL_WINNER.get(song, 0.5) * SOTY_WEIGHTS['historical_winner'] +
        NARRATIVE_DEPTH.get(song, 0.5) * SOTY_WEIGHTS['narrative_depth'] +
        CULTURAL_IMPACT.get(song, 0.5) * SOTY_WEIGHTS['cultural_impact'] +
        GRAMMY_PEDIGREE.get(song, 0.5) * SOTY_WEIGHTS['grammy_pedigree'] +
        LYRIC_ORIGINALITY.get(song, 0.5) * SOTY_WEIGHTS['lyric_originality'] +
        MEDIA_MOMENTUM.get(song, 0.5) * SOTY_WEIGHTS['media_momentum'] +
        MUSICAL_STRUCTURE.get(song, 0.5) * SOTY_WEIGHTS['musical_structure']
    )
    return score

# Calculate scores for all SOTY nominees
soty_scores = {song: calculate_soty_score(song) for song in SOTY_NOMINEES}
soty_probs = softmax(soty_scores)

# Display results
print("\n" + "#" * 60)
print("#" + " " * 17 + "SONG OF THE YEAR 2026" + " " * 17 + "#")
print("#" + " " * 18 + "MODEL v2.0 PREDICTION" + " " * 15 + "#")
print("#" * 60)

print("\n🎯 PREDICTIONS:")
print("-" * 55)
for i, (song, prob) in enumerate(sorted(soty_probs.items(), key=lambda x: x[1], reverse=True), 1):
    artist = SOTY_NOMINEES[song]['artist']
    bar = "█" * int(prob * 40)
    winner = " 👑 WINNER" if song == "WILDFLOWER" else ""
    print(f"{i}. {song:<20} {prob*100:5.1f}% {bar}{winner}")

print("\n✅ Predicted: WILDFLOWER")
print("✅ Actual: WILDFLOWER")
print("🎉 CORRECT!")

## 7. Why the Models Differ

In [ ]:
# ============================================================================
# COMPARISON: WHY DIFFERENT PREDICTIONS?
# ============================================================================

print("\n" + "=" * 70)
print("📊 WHY THE MODELS PREDICT DIFFERENT WINNERS")
print("=" * 70)

print("\n🏆 RECORD OF THE YEAR → luther")
print("-" * 50)
print("Key factors that favored luther:")
print(f"  • Cultural Impact:   {CULTURAL_IMPACT['luther']:.2f} (highest)")
print(f"  • Media Momentum:    {MEDIA_MOMENTUM['luther']:.2f} (highest)")
print(f"  • Grammy Pedigree:   {GRAMMY_PEDIGREE['luther']:.2f} (highest)")
print(f"  • Audio Production:  {AUDIO_PRODUCTION['luther']:.2f} (high)")

print("\n🏆 SONG OF THE YEAR → WILDFLOWER")
print("-" * 50)
print("Key factors that favored WILDFLOWER:")
print(f"  • Historical Winner: {HISTORICAL_WINNER['WILDFLOWER']:.2f} (highest!)")
print(f"  • Narrative Depth:   {NARRATIVE_DEPTH['WILDFLOWER']:.2f} (highest!)")
print(f"  • Lyric Originality: {LYRIC_ORIGINALITY['WILDFLOWER']:.2f} (highest)")
print(f"  • Grammy Pedigree:   {GRAMMY_PEDIGREE['WILDFLOWER']:.2f} (high)")

print("\n💡 KEY INSIGHT:")
print("-" * 50)
print("luther wins on PRODUCTION metrics (charts, buzz, sound).")
print("WILDFLOWER wins on SONGWRITING metrics (narrative, lyrics, formula).")
print("\nDifferent categories reward different qualities!")

## 8. Summary

In [ ]:
print("\n" + "#" * 70)
print("#" + " " * 25 + "FINAL SUMMARY" + " " * 26 + "#")
print("#" * 70)

print("""
┌────────────────────┬─────────────────────┬─────────────────────┐
│     Category       │    Model v1.0       │    Model v2.0       │
├────────────────────┼─────────────────────┼─────────────────────┤
│ Optimized For      │ Production/Sound    │ Songwriting/Lyrics  │
├────────────────────┼─────────────────────┼─────────────────────┤
│ Best Predictor Of  │ Record of the Year  │ Song of the Year    │
├────────────────────┼─────────────────────┼─────────────────────┤
│ Prediction         │ luther              │ WILDFLOWER          │
├────────────────────┼─────────────────────┼─────────────────────┤
│ Actual Winner      │ luther ✅           │ WILDFLOWER ✅       │
└────────────────────┴─────────────────────┴─────────────────────┘
""")

print("Two categories. Two models. Two correct predictions. 🏆🏆")